In [1]:
import os
os.system("sudo apt-get update")
os.system("sudo apt-get install -y python3-opencv")
os.system("pip install opencv-python")
os.system("pip install matplotlib")
os.system("pip install pandas")
import sys
sys.path.append("..")
import cv2
import numpy as np
from utils.imutils import jimshow as show
from utils.imutils import jimshow_channel as show_channel
import matplotlib.pyplot as plt
import pandas as pd

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease


Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
python3-opencv is already the newest version (4.5.4+dfsg-9ubuntu4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
#function for reading in a file and normailzing it
def normit(image):
    hist = cv2.calcHist([image],[0,1,2],None,[255,255,255],[0,256,0,256,0,256])
    normalized_hist = cv2.normalize(hist, hist, 0, 1.0, cv2.NORM_MINMAX)
    return(normalized_hist)

# loop load all flowers and names and save it
fp = "/work/cds-vis-data/flowers/"
fp_s = os.listdir(fp)


image_list = []
name_list = []
for i in fp_s:
    filepath = fp + i
    image = cv2.imread(filepath)
    names = i
    name_list.append(i)
    image_list.append(image)





In [15]:
#set filename
filename = 'image_1087.jpg'
#find according file index
target_ind = name_list.index(filename) 
#create target comparison hist
target = normit(image_list[target_ind])

#compare to target
comp_list =[]
for i in image_list[:100]:
    norm =  normit(i)
    histcomp = round(cv2.compareHist(target,norm,cv2.HISTCMP_CHISQR), 2)
    #save comparison values
    comp_list.append(histcomp)

#assign filenames to comparison values

df = pd.

In [9]:

#target by filename


target = normit(image_list[0])
comp_list =[]
for i in image_list[:100]:
    norm =  normit(i)
    histcomp = round(cv2.compareHist(target,norm,cv2.HISTCMP_CHISQR), 2)
    comp_list.append(histcomp)
    


In [11]:
comp_list.sort(reverse=True)
comp_list

[94560.35,
 85496.62,
 85481.48,
 82329.62,
 74353.93,
 71551.43,
 63249.66,
 58606.75,
 55558.33,
 55548.78,
 54081.15,
 50894.26,
 48701.57,
 48194.57,
 45536.38,
 43693.65,
 43687.8,
 42001.81,
 40747.98,
 40494.96,
 40164.35,
 39702.41,
 39641.63,
 38539.9,
 38434.25,
 37352.82,
 36422.96,
 36024.61,
 35649.48,
 34773.59,
 34515.07,
 33937.77,
 33652.13,
 33276.96,
 30620.34,
 29099.8,
 29062.7,
 28036.92,
 26921.82,
 26718.6,
 26605.53,
 25447.85,
 25346.58,
 25105.91,
 24713.84,
 24613.45,
 24457.36,
 24329.56,
 24282.75,
 23153.08,
 22084.43,
 22045.73,
 21670.2,
 21473.84,
 21058.08,
 21049.45,
 20832.18,
 19502.77,
 19501.1,
 19488.19,
 19312.14,
 19182.73,
 19026.91,
 19013.51,
 18976.7,
 18891.96,
 18853.35,
 18257.41,
 18233.9,
 18176.4,
 17824.27,
 17771.78,
 17641.46,
 17607.48,
 17185.76,
 17181.7,
 16927.31,
 16893.36,
 16654.96,
 16387.0,
 16237.52,
 16106.34,
 16039.2,
 16018.62,
 15921.0,
 15704.88,
 15487.47,
 15448.32,
 15099.72,
 15052.05,
 14996.0,
 14987.16,
 14

In [19]:
import pandas as pd

ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.